In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import metrics
import collection
import seaborn as sns

In [ ]:
obs=pd.read_csv('../data/box_samples_total.csv',index_col=0)
obs.describe(include='all')

In [ ]:
metrics.int_metric(obs.z,10,4)

10 million obs give 500k that intersect, and 116k with an I_4 metric

In [ ]:
bounds=[np.floor(obs.x.min()),np.floor(obs.y.min()),np.ceil(obs.x.max()-1e-3),np.ceil(obs.y.max())]

In [ ]:
bounds

In [ ]:
observations_plot=collection.create_image(obs.x,obs.y,np.ones_like(obs.x),0.5,bounds,mean=False)
np.nanmean(observations_plot)/np.nanstd(observations_plot)

In [ ]:
plt.imshow(observations_plot)

In [ ]:
pd.DataFrame(observations_plot.flatten()).describe()

In [ ]:
plt.hist(observations_plot.flatten())
plt.show()

Normally distributed number of observations in each cell

In [ ]:
intersection_plot=collection.create_image(obs.x,obs.y,~obs.z.isna(),0.5,bounds,mean=True)

In [ ]:
pd.DataFrame(intersection_plot.flatten()).describe()

In [ ]:
sns.histplot(intersection_plot.flatten())

In [ ]:
a=np.log(intersection_plot)
a=np.ma.masked_invalid(a)
sns.histplot(a.flatten())

lognormal distribution of intersection

In [ ]:
plt.imshow(a)

Spatial correlation of observation effectiveness. Being closer is better. Directly south is bad, but se and sw is good.

In [ ]:
plt.plot((1-obs.z.isna()).groupby(pd.cut(obs.d_building,100)).mean())

In [ ]:
plt.plot((1-obs.z.isna()).groupby(pd.cut(np.log(1+obs.d_building),100)).mean())

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
obs['i']=1-obs.z.isna()
obs

In [ ]:
m2=smf.glm('i ~ I(np.log(1+d_building))',data =obs,family=sm.families.Binomial()).fit()
print(m2.summary())

In [ ]:
p=obs.i.groupby(pd.cut(np.log(1+obs.d_building),100,include_lowest=True)).mean()
odds=p/(1-p)
odds

In [ ]:
y=np.exp(m2.params[0]+ m2.params[1]*(odds.index.categories.mid))
p=y/(1+y)
log_predicts=pd.DataFrame({"x":np.exp(odds.index.categories.mid)-1,"y":y,"p":p})
log_predicts

In [ ]:
(odds/(1+odds))

In [ ]:
a= sns.scatterplot(np.exp(odds.index.categories.mid)-1,odds/(1+odds))
sns.lineplot(x='x',y='p',data=log_predicts,ax=a)
# plt.yscale('log')
plt.xscale('log')
plt.xlim(1,100)
plt.ylabel('Probability')
plt.xlabel('Distance (metres)')
plt.savefig('../figures/distance_regression.png')


In [ ]:
intersection_residuals_plot=collection.create_image(obs.x.round(0),obs.y.round(0),obs.i-m2.fittedvalues,1,bounds,mean=True)

In [ ]:
pd.DataFrame(intersection_residuals_plot.flatten()).describe()

In [ ]:
a=plt.imshow(intersection_residuals_plot,cmap='coolwarm',vmin=-0.15,vmax=0.15,origin='lower')
plt.colorbar(a)
plt.savefig('../figures/distance_residuals.png')

We also need to consider the effect of building height.


In [ ]:
sns.histplot(np.tan(np.deg2rad(obs.el)),stat='probability',bins=25,binrange=(0,2.5))
plt.xlabel('Intersection height as a fraction of distance')
plt.savefig('../figures/elevation.png')

In [ ]:
np.tan(np.deg2rad(obs.el))

In [ ]:
from metrics import _indicator as indicator

In [ ]:
cuts_=pd.cut(obs.d_building,100)

In [ ]:
indicators = []
for i,h in enumerate([5,10,20,40]):
    for j,m in enumerate([1,4,8,np.inf]):
        df=pd.DataFrame({'Height':h,'Metric':m, 'Distance': cuts_.cat.categories.mid, "Proportion": indicator(obs.z,h,m).astype('int').groupby(cuts_).mean()})
        indicators.append(df)
indicators=pd.concat([i.reset_index() for i in indicators])
indicators.Height=indicators.Height.astype('category')
indicators.Metric=indicators.Metric.astype('category')

In [ ]:
g=sns.FacetGrid(indicators,col='Height',col_wrap=2,sharex=True,sharey=True,legend_out=True)
g=g.map_dataframe(sns.lineplot,x='Distance',y='Proportion',hue='Metric').add_legend()
g._legend.set_title('Metrics')
new_labels = ['I_1', 'I_4','I_8', 'I_inf']
for t, l in zip(g._legend.texts, new_labels): t.set_text(l)
plt.xscale('log')
plt.yscale('log')
g.set_ylabels('Probability')
g.set_xlabels('Distance')
# plt.legend()
plt.savefig('../figures/height_metrics.png')

In [ ]:
fig,axes=plt.subplots(ncols=4,figsize=(12,4))
for i,h in enumerate([10,20,30,40]):
    axes[i].imshow(collection.create_image(obs.x,obs.y,indicator(obs.z,h,4).astype('int'),0.25,bounds,mean=False)/(i_count+0.01),vmin=0,vmax=0.35)

In [ ]:
fig,axes=plt.subplots(ncols=4,figsize=(12,4))
for i,h in enumerate([10,20,30,40]):
    axes[i].imshow(collection.create_image(obs.x,obs.y,indicator(obs.z,h,4).astype('int'),0.5,bounds,mean=True),vmin=0,vmax=0.01)